In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from catboost import CatBoostRegressor, Pool

# import torch
# import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_ace_24 = pd.read_csv("Ace_погружение_24часа.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_24 = pd.read_csv("Discover_погружение_24часа_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_ace_autocorr = pd.read_csv("Ace_погружение_автокорр.csv", sep=';', decimal=',', parse_dates=['datetime'])
data_discover_autocorr = pd.read_csv("Discover_погружение_автокорр_до_22.09.2024.csv", sep=';', decimal=',', parse_dates=['datetime'])

In [3]:
datasets = [data_ace_24, data_discover_24, data_ace_autocorr, data_discover_autocorr]

In [4]:
for dataset in datasets:
    dataset.set_index('datetime', inplace=True)
    dataset.dropna(inplace=True)
    if not dataset.index.is_monotonic_increasing:
        dataset.sort_index(inplace=True)

In [5]:
from_index_dst_data = data_ace_24.columns.get_loc('Dst') #индекс, с которого начинаются данные о нынешнем и предыдущих значениях Dst, величине ММП и параметрах СВ
hour_horizont = 3
target_columns = ['Dst +1', 'Dst +2', 'Dst +3']

### Модели

In [6]:
model_cat = CatBoostRegressor(iterations=500,
                              depth=2,
                              learning_rate=1,
                              loss_function='RMSE',
                              logging_level='Silent',
                              early_stopping_rounds=40)

pipe = make_pipeline(StandardScaler(), LinearRegression())

In [7]:
def linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for i in range(len(y_trains)):
        pipe.fit(x_train, y_trains[i])
        y_pred = pipe.predict(x_test)

        r_2 = metrics.r2_score(y_tests[i], y_pred)
        mse = metrics.mean_squared_error(y_tests[i], y_pred)

        # print(f'Прогноз на {i+1} час:')
        # print(f'MSE={mse} и r^2={r_2} линейная регрессия, погружение - {depth}')

        new_row = {
            'Способ обучения': type_train_test,
            'Глубина погружения': depth,
            'Модель': 'Линейная регрессия',
            'Горизонт предсказания': f'Dst +{i+1}',
            'R_2 score': r_2,
            'MSE': mse
        }
        results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
    return results

In [8]:
def linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, lambda_values, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for lambda_val in lambda_values:
        for i in range(len(y_trains)):
            ridge_reg = Ridge(lambda_val)
            ridge_reg.fit(x_train, y_trains[i])
            y_pred = ridge_reg.predict(x_test)
            
            r_2 = metrics.r2_score(y_tests[i], y_pred)
            mse = metrics.mean_squared_error(y_tests[i], y_pred)
            
            # print(f'Прогноз на {i+1} час:')
            # print(f'MSE={mse} и r^2={r_2} с Lambda={lambda_val} линейная регрессия с Ridge-регуляризацией, погружение - {depth}')

            new_row = {
                'Способ обучения': type_train_test,
                'Глубина погружения': depth,
                'Модель': f'Линейная регрессия c регуляризацией lambda={lambda_val}',
                'Горизонт предсказания': f'Dst +{i+1}',
                'R_2 score': r_2,
                'MSE': mse
            }

            results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
    return results

In [9]:
def catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, depth, results):
    y_trains = [y_train_1, y_train_2, y_train_3]
    y_tests = [y_test_1, y_test_2, y_test_3]

    for i in range(len(y_trains)):
        train_pool = Pool(x_train, y_trains[i], cat_features=[])
        test_pool = Pool(x_test, y_tests[i], cat_features=[])

        model_cat.fit(train_pool)
        y_pred = model_cat.predict(test_pool)

        r_2 = metrics.r2_score(y_tests[i], y_pred)
        mse = metrics.mean_squared_error(y_tests[i], y_pred)

        # print(f'Прогноз на {i+1} час:')
        # print(f'MSE={mse} и r^2={r_2}  CatBoost, погружение - {depth}')

        new_row = {
            'Способ обучения': type_train_test,
            'Глубина погружения': depth,
            'Модель': 'CatBoost',
            'Горизонт предсказания': f'Dst +{i+1}',
            'R_2 score': r_2,
            'MSE': mse
        }

        results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)  
    return results

#### Разбиения данных на тренировочные и тестовые наборы

In [10]:
def split_data(data, target_columns, test_size=0.15, shuffle=False):
    y_train = {i+1: None for i in range(hour_horizont)}
    y_test = {i+1: None for i in range(hour_horizont)}
    for i, target_col in enumerate(target_columns, start=1):
        x_train, x_test, y_train[i], y_test[i] = train_test_split(data.iloc[:, from_index_dst_data:], data[target_col], test_size=test_size, shuffle=shuffle)
    return x_train, x_test, y_train[1], y_test[1], y_train[2], y_test[2], y_train[3], y_test[3]

#### Создание пустого датафрейма для записи результатов

In [11]:
results = pd.DataFrame()
columns = ['Способ обучения', 'Глубина погружения', 'Модель', 'Горизонт предсказания', 'R_2 score', 'MSE']

results = pd.DataFrame(columns=columns)

### Обучение на данных ACE, тестирование на данных ACE

In [12]:
type_train_test = 'ACE-ACE'

#### Погружение - 24 часа по всем переменным

In [13]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_ace_24, target_columns)

In [14]:
lambda_values = [1000000, 5000000, 10000000]

In [15]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)

#### Погружение с учётом автокорреляционной функции

In [16]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_ace_autocorr, target_columns)

In [17]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)

### Обучение на данных DSCOVR, тестирование на данных DSCOVR

In [18]:
type_train_test = 'DSCOVR-DSCOVR'

#### Погружение - 24 часа по всем переменным

In [19]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_24, target_columns)

In [20]:
lambda_values = [0.01, 10]

In [21]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)

#### Погружение с учётом автокорреляционной функции

In [22]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_autocorr, target_columns)

In [23]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)

### Обучение на данных ACE, тестирование на данных DSCOVR

In [24]:
type_train_test = 'ACE-DSCOVR'

#### Погружение - 24 часа по всем переменным

In [25]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_24, target_columns)
y_test_disc_cross = {}

common_index = y_test_1.index.unique().intersection(data_discover_24.index.unique())
x_test_disc_cross = data_discover_24.loc[common_index].iloc[:, from_index_dst_data:]

for i in range(1, 4):
    y_test_disc_cross[i] = data_discover_24[f'Dst +{i}'].loc[common_index]

In [26]:
x_train, x_test, y_train_1, y_test_1 = x_train, x_test_disc_cross, y_train_1, y_test_disc_cross[1]
y_train_2, y_test_2 = y_train_2, y_test_disc_cross[2]
y_train_3, y_test_3 = y_train_3, y_test_disc_cross[3]

In [27]:
lambda_values = [500000, 1000000, 5000000]

In [28]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)

#### Погружение с учётом автокорреляционной функции

In [30]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_autocorr, target_columns)
y_test_disc_cross = {}

common_index = y_test_1.index.unique().intersection(data_discover_24.index.unique())
x_test_disc_cross = data_discover_autocorr.loc[common_index].iloc[:, from_index_dst_data:]

for i in range(1, 4):
    y_test_disc_cross[i] = data_discover_autocorr[f'Dst +{i}'].loc[common_index]

In [31]:
x_train, x_test, y_train_1, y_test_1 = x_train, x_test_disc_cross, y_train_1, y_test_disc_cross[1]
y_train_2, y_test_2 = y_train_2, y_test_disc_cross[2]
y_train_3, y_test_3 = y_train_3, y_test_disc_cross[3]

In [32]:
lambda_values = [500000, 1000000, 5000000]

In [33]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)

### Обучение на данных DSCOVR, тестирование на данных ACE

In [34]:
type_train_test = 'DSCOVR-ACE'

#### Погружение - 24 часа по всем переменным

In [35]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_24, target_columns)
y_test_ace_cross = {}

common_index = y_test_1.index.unique().intersection(data_ace_24.index.unique())
x_test_ace_cross = data_ace_24.loc[common_index].iloc[:, from_index_dst_data:]

for i in range(1, 4):
    y_test_ace_cross[i] = data_ace_24[f'Dst +{i}'].loc[common_index]

In [36]:
lambda_values = [500000, 1000000, 5000000]

In [37]:
x_train, x_test, y_train_1, y_test_1 = x_train, x_test_ace_cross, y_train_1, y_test_ace_cross[1]
y_train_2, y_test_2 = y_train_2, y_test_ace_cross[2]
y_train_3, y_test_3 = y_train_3, y_test_ace_cross[3]

In [38]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, '24 часа', results)

#### Погружение с учётом автокорреляционной функции

In [39]:
x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3 = split_data(data_discover_autocorr, target_columns)
y_test_ace_cross = {}

common_index = y_test_1.index.unique().intersection(data_ace_autocorr.index.unique())
x_test_ace_cross = data_ace_autocorr.loc[common_index].iloc[:, from_index_dst_data:]

for i in range(1, 4):
    y_test_ace_cross[i] = data_ace_autocorr[f'Dst +{i}'].loc[common_index]

In [40]:
lambda_values = [500000, 1000000, 5000000]

In [41]:
x_train, x_test, y_train_1, y_test_1 = x_train, x_test_ace_cross, y_train_1, y_test_ace_cross[1]
y_train_2, y_test_2 = y_train_2, y_test_ace_cross[2]
y_train_3, y_test_3 = y_train_3, y_test_ace_cross[3]

In [42]:
results = linear_regression(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)
results = linear_regression_ridge(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', lambda_values, results)
results = catboost(x_train, x_test, y_train_1, y_test_1, y_train_2, y_test_2, y_train_3, y_test_3, 'автокорреляционная функция', results)

### ТРИВИАЛЬНАЯ МОДЕЛЬ

#### ACE на ACE

In [44]:
x_train, x_test, y_train, y_test = \
train_test_split(data_ace_autocorr.iloc[:, from_index_dst_data:], data_ace_autocorr['Dst +1'], test_size=0.15, shuffle=False)

In [45]:
r_2 = metrics.r2_score(y_test, x_test['Dst'])
mse = metrics.mean_squared_error(y_test, x_test['Dst'])

print(f'MSE={mse} и r^2={r_2} линейная регрессия, погружение - автокорреляционная функция')

MSE=9.911526827985272 и r^2=0.9272754951721692 линейная регрессия, погружение - автокорреляционная функция


#### DSCVR на DSCVR

In [46]:
x_train, x_test, y_train, y_test = \
train_test_split(data_discover_autocorr.iloc[:, from_index_dst_data:], data_discover_autocorr['Dst +1'], test_size=0.15, shuffle=False)

In [47]:
r_2 = metrics.r2_score(y_test, x_test['Dst'])
mse = metrics.mean_squared_error(y_test, x_test['Dst'])

print(f'MSE={mse} и r^2={r_2} линейная регрессия, погружение - автокорреляционная функция')

MSE=12.148033924441018 и r^2=0.925786722657774 линейная регрессия, погружение - автокорреляционная функция


In [48]:
results

,Способ обучения,Глубина погружения,Модель,Горизонт предсказания,R_2 score,MSE
0,ACE-ACE,24 часа,Линейная регрессия,Dst +1,-0.871937,256.580835
1,ACE-ACE,24 часа,Линейная регрессия,Dst +2,-0.998146,273.882736
2,ACE-ACE,24 часа,Линейная регрессия,Dst +3,-0.418394,194.427491
3,ACE-ACE,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +1,0.710778,39.642872
4,ACE-ACE,24 часа,Линейная регрессия c регуляризацией lambda=100...,Dst +2,0.636495,49.825048
...,...,...,...,...,...,...
109,DSCOVR-ACE,автокорреляционная функция,Линейная регрессия c регуляризацией lambda=500...,Dst +2,0.343024,107.608014
110,DSCOVR-ACE,автокорреляционная функция,Линейная регрессия c регуляризацией lambda=500...,Dst +3,0.067609,152.742920
111,DSCOVR-ACE,автокорреляционная функция,CatBoost,Dst +1,0.928750,11.669494
112,DSCOVR-ACE,автокорреляционная функция,CatBoost,Dst +2,0.835959,26.868794


In [49]:
results.to_excel('результаты обучения без адаптации.xlsx', index=False)

In [50]:
results.to_csv('результаты обучения без адаптации.csv', sep=';', encoding='utf-8')